In [1]:
import tensorflow.keras as keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import logging
import os
import pandas as pd
from matplotlib import pyplot
import nngp
import neural_net
import random
from sklearn.model_selection import train_test_split
import bokeh.io
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score

from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import row, gridplot
from bokeh.palettes import Category10
from bokeh.models import Legend
bokeh.io.reset_output()
bokeh.io.output_notebook()

logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)

%load_ext autoreload
%autoreload 2

Loading BokehJS ...

## Experimental Functions

In [2]:
def prep_data(X, Y, dtype=tf.float64):
    X_flat = tf.convert_to_tensor(X, dtype=dtype)
    Y_cat = keras.utils.to_categorical(Y)
    Y_cat = Y_cat - 0.1
    Y_cat = tf.convert_to_tensor(Y_cat, dtype=dtype)
    return X_flat, Y_cat

In [3]:
def evaluate_NN(X_train, Y_train, X_test, Y_test, model, training_accuracy=1, max_epochs=400):
    
    callbacks = AccuracyCallback(training_accuracy, max_epochs)
    model.fit(X_train, Y_train, verbose=0, callbacks=[callbacks], epochs=max_epochs)
    output = model.predict(X_test, verbose=0)
    loss, acc = model.evaluate(X_test, Y_test, verbose=0)
    _, training_acc = model.evaluate(X_train, Y_train, verbose=0)
    
    return acc, loss, training_acc

In [4]:
class AccuracyCallback(tf.keras.callbacks.Callback): 
    def __init__(self, accuracy_threshold, max_epoch):
        self.accuracy_threshold = accuracy_threshold
        self.max_epoch = max_epoch
        super(AccuracyCallback, self).__init__()
    
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('categorical_accuracy') >= self.accuracy_threshold):   
            print("Reached %2.1f%% accuracy, so stopping training \n" %(self.accuracy_threshold*100))   
            self.model.stop_training = True
        if epoch == self.max_epoch:
            print(f"Stopping training after {epoch} epochs, final accuracy: {logs.get('categorical_accuracy')}")

In [5]:
def evaluate_GP(X_train, Y_train, X_test, Y_test, K):
    mu_bar, _ = nngp.GP_cholesky(X_train, Y_train, X_test, K)

    # Calculate Accuracy
    predictions = tf.argmax(mu_bar, axis=1).numpy()
    predictions_true = tf.argmax(Y_test, axis=1).numpy()
    acc = accuracy_score(predictions_true,predictions)
    
    # MSE Loss
    loss = tf.keras.losses.MSE(mu_bar, Y_test)
    average_loss = loss.numpy().mean()
    return acc, average_loss

In [6]:
def save_bokeh(plot, filepath):
    bokeh.io.reset_output()
    bokeh.io.output_file(filepath, plot)
    show(p)
    bokeh.io.reset_output()
    bokeh.io.output_notebook()

### Neural Networks vs NNGP on MNIST

In [7]:
def df_append(df, data):
    df.loc[len(df)] = data

In [ ]:
#New parameters 
alpha_b = 4
beta_b = 2
shape_b, scale_b = alpha_b, 1/beta_b
var_b = 1/np.random.gamma(shape_b, scale_b)

In [ ]:
alpha_w = 4
beta_w = 4
shape_w, scale_w = alpha_w, 1/beta_w
var_w = 1/np.random.gamma(shape_w, scale_w)

In [8]:
def run_over_dataset_and_widths(X_train_flat, Y_train_reg, X_test_flat, Y_test_reg, activations):

    widths = [5, 10, 50, 100, 500, 1000]
    dataset_size = [75, 150, 250, 436]
    n_layers = 3

    df_columns = ['activation', 'dataset_size', 'width', 'test_accuracy', 'test_mse', 'training_accuracy']
    nn_evaluate = pd.DataFrame(columns=df_columns)
    gp_evaluate = pd.DataFrame(columns=df_columns)
    gpo_evaluate = pd.DataFrame(columns=df_columns)
    
    #Test
    X_test_n, Y_test_n =  X_test_flat, Y_test_reg
        
    for act in activations:
        
        # Kernel settings don't change
        sigma_b = np.sqrt(var_b)
        sigma_w = np.sqrt(var_w)
        general_kernel = nngp.GeneralKernel(act,
                                            L=n_layers,
                                            n_g=401,
                                            n_v=400,
                                            n_c=400,
                                            u_max=10,
                                            s_max=100,
                                            sigma_b=sigma_b,
                                            sigma_w=sigma_w)
       #Original
        general_kernel2 = nngp.GeneralKernel(act,
                                            L=n_layers,
                                            n_g=401,
                                            n_v=400,
                                            n_c=400,
                                            u_max=10,
                                            s_max=100,
                                            sigma_b=np.sqrt(0.1),
                                            sigma_w=np.sqrt(1.6))

        for n, n_data in enumerate(dataset_size):
            print(f'Training Data set size of {n_data} with {act.__name__} activation')
            
            
            # Sizes for building nerual net
            din=len(X_train_flat[0])
            dout = len(Y_train_reg[0])
            
            # Construct the dataset size
            X_train_n, Y_train_n = train_random_sample(X_train_flat,Y_train_reg,n_data,din,dout)

            # Evaluate the Origianl GP first
            gpo_acc, gpo_loss = evaluate_GP(X_train_n, Y_train_n, X_test_n, Y_test_n, general_kernel2.K)
            df_append(gpo_evaluate, [act.__name__, n_data, np.nan, gpo_acc, gpo_loss, np.nan])
            
            # Evaluate the New GP 
            gp_acc, gp_loss = evaluate_GP(X_train_n, Y_train_n, X_test_n, Y_test_n, general_kernel.K)
            df_append(gp_evaluate, [act.__name__, n_data, np.nan, gp_acc, gp_loss, np.nan])

            # Evaluate the nn over a range of widths
            for i, width in enumerate(widths):
                print(f'Training model of width: {width} and depth: {n_layers}')
                model = neural_net.build_model((din,), dout, n_layers, width, hidden_activation=act, final_activation=None)
                model.compile(optimizer='Adam', loss='MSE', metrics=['categorical_accuracy'])
                nn_acc, nn_loss, nn_train_acc = evaluate_NN(X_train_n, Y_train_n, X_test_n, Y_test_n, model)
                df_append(nn_evaluate, [act.__name__, n_data, width, nn_acc, nn_loss, nn_train_acc])

    return gp_evaluate, gpo_evaluate, nn_evaluate

In [ ]:
#Load the Sample Entropy dataset
dataset = pd.read_csv('Sample_Entropy.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.15, random_state=0)

In [ ]:
#Prepare Data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train_flat, Y_train_reg = prep_data(X_train, Y_train)
X_test_flat, Y_test_reg = prep_data(X_test, Y_test)

In [9]:
def train_random_sample(X_train,Y_train,n_data,din,dout):
    S=tf.concat([X_train, Y_train],1)
    Random_S=tf.stack(random.sample(list(S),n_data),0)
    X_train_F=tf.split(Random_S,[din,dout],1)[0]
    Y_train_F=tf.split(Random_S,[din,dout],1)[1]
    return X_train_F,Y_train_F

In [ ]:
#Training all models 
activations = [tf.nn.tanh, tf.nn.relu]
gp_evaluate, gpo_evaluate, nn_evaluate = run_over_dataset_and_widths(X_train_flat, Y_train_reg, X_test_flat, Y_test_reg, activations)

In [ ]:
gp_evaluate.to_csv('experimental_data/gp_results.csv')
gpo_evaluate.to_csv('experimental_data/gpo_results.csv')
nn_evaluate.to_csv('experimental_data/nn_results.csv')

In [14]:
def plot_metric(metric, y_label, legend_loc='bottom_right', save_to_file=None):
    widths = nn_evaluate.width.unique()
    activations = ['relu', 'tanh']


    # create three plots
    x_label = 'Training Dataset Size'
    y_label = y_label
    legend=[]
    s1 = figure(title='Relative_Power-Data, Relu', x_axis_label = x_label, y_axis_label = y_label, plot_width=800, plot_height=400)
    s2 = figure(title='Relative_Power-Data, Tanh', x_axis_label = x_label)
    s3 = figure()

    color = Category10[len(widths)]

    gp_relu = gp_evaluate[gp_evaluate.activation=='relu']
    gp_tanh = gp_evaluate[gp_evaluate.activation=='tanh']
    s1.line(gp_relu.dataset_size, gp_relu[metric],color='indigo', line_dash=[3], line_width=2)
    r = s2.line(gp_relu.dataset_size, gp_relu[metric],color='indigo', line_dash=[3], line_width=2, legend_label='Gaussian Process')
    #legend.append(('Gaussian Process', [r]))

    for i, width in enumerate(widths):
        nn_relu = nn_evaluate[(nn_evaluate.width==width) & (nn_evaluate.activation=='relu')]
        nn_tanh = nn_evaluate[(nn_evaluate.width==width) & (nn_evaluate.activation=='tanh')]
        s1.line(nn_relu.dataset_size, nn_relu[metric], color=color[i], line_width=2)
        r = s2.line(nn_tanh.dataset_size, nn_tanh[metric], color=color[i], line_width=2, legend_label=f'NN width: {width}')
        legend.append((f'NN width: {width}', [r]))

    # make a grid
    grid = gridplot([s1, s2], ncols=2, plot_width=400, plot_height=350, sizing_mode='scale_both')

    # s2.legend = Legend(items=legend)
    s2.legend.location = legend_loc
    s2.legend.spacing = -10
    # s2.add_layout(legend, 'right')
    s2.legend.orientation = "vertical"
    s2.legend.label_text_font_size = "9px"
    s2.y_range.start = 0
    s1.y_range.start = 0

    show(grid)
    return grid, s1, s2

In [13]:
# Plot the Relative Power results
gp_evaluate = pd.read_csv('Results/experimental_dataRP/gp_results.csv')
gpo_evaluate = pd.read_csv('Results/experimental_dataRP/gpo_results.csv')
nn_evaluate = pd.read_csv('Results/experimental_dataRP/nn_results.csv')
p, s1, s2 = plot_metric('test_accuracy', 'Test Accuracy')

In [ ]:
save_bokeh(p, 'charts/mnist-acc.html')

In [15]:
p = plot_metric('test_mse', 'Mean Squared Error', legend_loc='top_right')

In [ ]:
save_bokeh(p, 'charts/mnist-loss.html')